In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

import vectorbt as vbt

from dffc.holt_winters._holt_winters import HW, HWDP

d:\anaconda3\envs\quant\Lib\site-packages\telegram\utils\request.py:54: UserWarning:

python-telegram-bot wasn't properly installed. Please refer to README.rst on how to properly install.



In [3]:
from dffc.fund_data import register_fund_data

register_fund_data()

FundData has been registered to vectorbt namespace


In [ ]:
try:
    ndgd_price = pd.read_pickle('ndgd_price.pkl')
except FileNotFoundError:
    ndgd_data = vbt.FundData.download(
        ['270042', '002963'],
        names=['ND', 'GD'],
        start='2022-01-01',
        end="2025-09-04"
    )
    import pickle

    ndgd_price = ndgd_data.get('unit_value')['270042'].dropna()
    ndgd_price.to_pickle('ndgd_price.pkl')
    # ndgd_data = ndgd_data.update(end="2025-09-01")


In [4]:
ndgd_data = vbt.FundData.download(
    ['270042', '002963'],
    names=['ND', 'GD'],
    start='2022-01-01',
    end="2025-09-04"
)

In [5]:
from dffc.holt_winters._optimization import process_hw_opt, process_single_hw_opt

In [7]:
prices = ndgd_data.get('unit_value')

In [ ]:
results = []
for col in range(prices.shape[1]):
    results.append(process_single_hw_opt(prices.to_numpy()[:, col], ".", 8))

In [ ]:
result = process_single_hw_opt(prices.to_numpy()[:, 0], ".", 8)

Batch optimizing:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
# hw = HW.run(ndgd_price[ndgd_price.index >= "2022-01-01"], alpha=0.030732963, beta=0.5, gamma=1, m=14)
hw = HW.run(
    ndgd_data.get('unit_value'), 
    alpha=0.5, # fit.params['smoothing_level'],
    beta=0.2, # fit.params['smoothing_trend'],
    gamma=1, # fit.params['smoothing_seasonal'],
    m=14, multiplicative=True
    )

In [ ]:
pd.concat([fit.fittedvalues, ndgd_price[ndgd_price.index >= "2022-01-01"]], axis=1).plot(figsize=(12, 6))